In [31]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import recall_score,accuracy_score,roc_auc_score,confusion_matrix,roc_curve






In [32]:
df_base=pd.read_excel('dataset_780.xlsx',sheet_name=0)

df_base=df_base.dropna(subset=['Diagnosis'])

mapping={
    'no':0,
    'yes':1,
    'male':1,
    'female':0,
    'normal':0,
    'diarrhea':1,
    'constipation':1,
    '+':1,
    '++':2,
    '+++':3,
    'local':1,
    'generalized':2,
    'appendicitis':1,
    'no appendicitis':0,
    'uncomplicated':0,
    'complicated':1,
}

for col in df_base.columns:
    df_base[col]=df_base[col].replace(mapping)


feat_all= ['Peritonitis','Ketones_in_Urine','Nausea','Free_Fluids','Loss_of_Appetite','Appendix_Diameter','CRP','WBC_Count','Neutrophil_Percentage','Diagnosis']

df=df_base[feat_all]
print (df)






     Peritonitis  Ketones_in_Urine  Nausea  Free_Fluids  Loss_of_Appetite  \
0            0.0               2.0     0.0          0.0               1.0   
1            0.0               0.0     0.0          0.0               1.0   
2            0.0               0.0     0.0          0.0               0.0   
3            0.0               0.0     1.0          0.0               1.0   
4            0.0               0.0     1.0          0.0               1.0   
..           ...               ...     ...          ...               ...   
777          1.0               NaN     0.0          0.0               1.0   
778          1.0               NaN     0.0          0.0               1.0   
779          0.0               0.0     0.0          1.0               0.0   
780          1.0               3.0     1.0          1.0               0.0   
781          1.0               0.0     0.0          0.0               0.0   

     Appendix_Diameter    CRP  WBC_Count  Neutrophil_Percentage  Diagnosis 

/var/folders/gt/jkhq8fjj7rg_v80lhs4scmh80000gn/T/ipykernel_2229/2790724999.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_base[col]=df_base[col].replace(mapping)


In [33]:

x = df.iloc[:, 0:-1]  
y = df.iloc[:, -1]



x_temp,x_test, y_temp,y_test = train_test_split(x,y,test_size=0.2,random_state=88)
x_train,x_val,y_train,y_val = train_test_split(x_temp,y_temp,test_size=0.25,random_state=88)

In [34]:
model_xgb=xgb.XGBClassifier(
    n_estimators=222,
    max_depth=3,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=17,
    eval_metric='logloss',
    reg_alpha=0.1,
    reg_lambda=0.1,
)

In [35]:
model_xgb.fit(x_train, y_train)
scores = cross_val_score(model_xgb, x_train, y_train, cv=10, scoring='accuracy', n_jobs=2)
print("Mean cross_val accuracy Score:", scores.mean())
print("Standard Deviation of cross_val accuracy Scores:", scores.std())

scores_auc = cross_val_score(model_xgb, x_train, y_train, cv=10, scoring='roc_auc', n_jobs=2)
print("Mean cross_val auc-roc score:", scores_auc.mean())
print("Standard Deviation of auc-roc Scores:", scores_auc.std())

scores_recall = cross_val_score(model_xgb, x_train, y_train, cv=10, scoring='recall', n_jobs=2)
print("Mean cross_val recall score:", scores_recall.mean())
print("Standard Deviation of recall Scores:", scores_recall.std())

Mean cross_val accuracy Score: 0.8909343200740055
Standard Deviation of cross_val accuracy Scores: 0.022801412132471593
Mean cross_val auc-roc score: 0.9594082428292954
Standard Deviation of auc-roc Scores: 0.023121778017022976
Mean cross_val recall score: 0.9084656084656085
Standard Deviation of recall Scores: 0.04120449989464045


In [36]:


y_pred_prob=model_xgb.predict_proba(x_val)[:,1]
fpr, tpr, thresholds = roc_curve(y_val, y_pred_prob)
youden_index = tpr - fpr
optimal_idx = np.argmax(youden_index)
optimal_threshold = thresholds[optimal_idx]

distances = np.sqrt((1 - tpr)**2 + fpr**2)
optimal_idx_eu = np.argmin(distances)
optimal_threshold_eu = thresholds[optimal_idx_eu]

print(f"Optimal Threshold Youden: {optimal_threshold} and Optimal Euclidean Distance :{optimal_threshold_eu}")

threshold_y = optimal_threshold
threshold_eu = 0.8


y_val_prob=model_xgb.predict_proba(x_val)[:,1]
y_thres=(y_val_prob >= threshold_y).astype(int)
auc_score = roc_auc_score(y_val, y_val_prob)
print("Val AUC:", auc_score)
val_conf_matrix = confusion_matrix(y_val, y_thres)
TNv, FPv, FNv, TPv = val_conf_matrix.ravel()
val_sensitivity = TPv / (TPv + FNv)
val_specificity = TNv / (TNv + FPv)
val_ppv = TPv/(TPv+FPv)
val_npv = TNv/(TNv+FNv)

print("For Youden Index")

print(f"Sensitivity: {val_sensitivity:.3f} ({val_sensitivity * 100:.2f}%)")
print(f"Specificity: {val_specificity:.3f} ({val_specificity * 100:.2f}%)")

print (f"Positive predictive value : {val_ppv:.3f}({val_ppv*100:.2f}%)")
print (f"Negative predictive value : {val_npv:.3f}({val_npv*100:.2f}%)")

print("Val Confusion Matrix:\n", val_conf_matrix)
val_acu=accuracy_score(y_val,y_thres)
print("Val accuracy score:",val_acu)

y_val_prob=model_xgb.predict_proba(x_val)[:,1]
y_thres=(y_val_prob >= threshold_eu).astype(int)
auc_score = roc_auc_score(y_val, y_val_prob)
print("Val AUC:", auc_score)
val_conf_matrix = confusion_matrix(y_val, y_thres)
TNv, FPv, FNv, TPv = val_conf_matrix.ravel()
val_sensitivity = TPv / (TPv + FNv)
val_specificity = TNv / (TNv + FPv)
val_ppv = TPv/(TPv+FPv)
val_npv = TNv/(TNv+FNv)

print("For Eucledean distance")

print(f"Sensitivity: {val_sensitivity:.3f} ({val_sensitivity * 100:.2f}%)")
print(f"Specificity: {val_specificity:.3f} ({val_specificity * 100:.2f}%)")

print (f"Positive predictive value : {val_ppv:.3f}({val_ppv*100:.2f}%)")
print (f"Negative predictive value : {val_npv:.3f}({val_npv*100:.2f}%)")

print("Val Confusion Matrix:\n", val_conf_matrix)
val_acu=accuracy_score(y_val,y_thres)
print("Val accuracy score:",val_acu)


Optimal Threshold Youden: 0.9569246172904968 and Optimal Euclidean Distance :0.9152830839157104
Val AUC: 0.9447811447811448
For Youden Index
Sensitivity: 0.844 (84.44%)
Specificity: 0.939 (93.94%)
Positive predictive value : 0.950(95.00%)
Negative predictive value : 0.816(81.58%)
Val Confusion Matrix:
 [[62  4]
 [14 76]]
Val accuracy score: 0.8846153846153846
Val AUC: 0.9447811447811448
For Eucledean distance
Sensitivity: 0.889 (88.89%)
Specificity: 0.864 (86.36%)
Positive predictive value : 0.899(89.89%)
Negative predictive value : 0.851(85.07%)
Val Confusion Matrix:
 [[57  9]
 [10 80]]
Val accuracy score: 0.8782051282051282
